<a href="https://colab.research.google.com/github/jvataidee/DataAnalysis/blob/master/Analise_Financeira_Mercado_Gado_de_Corte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**by: [João Ataíde](https://www.joaoataide.com)**
# Análise Mercado Financeira de Gado de Corte


---


## Importanto bibliotecas básicas

In [1]:
#importar bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# configurar a visualização
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Obtenção dos dados

Encontrar dados para começar as análises sempre é um processo complicado, em especial quando a gente está falando de um eixo de mercado ainda muito artesanal, o qual etá modificando agora seu modo com a Pecuária 4.0, assim, não foi possível encontrar uma API nacional que automatize a obtenção dados sobre o preço das commodities.

Então, para isso realizei a instalação de duas bibliotecas investpy, API criada a parti da plataforma [investing.com](https://investing.com/) que possui diversos bases de dados financeiras, não somente de commodities. Outra biblioteca que foi necessária foi a [Yahoo! Finance](https://finance.yahoo.com/) a qual utilizei para obter os dados referentes a cotas do dollar.


In [ ]:
!pip install investpy -q

In [ ]:
!pip install yfinance -q

In [4]:
pip install -U sidrapy -q

In [5]:
#importar bibliotecas financeiras
import investpy
import yfinance as yf
import sidrapy

### Dados de Commodities

In [87]:
#Lista de commodities
commodities = ["Live Cattle", "Feeder Cattle", ]

#Data de inicio e fim de análise
inicio = "01/01/2014"
fim = "31/12/2020"

In [88]:
#Função para importar dados das commodities
def commodities_df(commodities, inicio,fim):
  precos = pd.DataFrame()
  for i in commodities:
    df = investpy.commodities.get_commodity_historical_data(commodity=i, from_date=inicio, to_date=fim).Close
    df.rename(i, inplace=True)
    df.columns = [i]
    precos = pd.concat([precos,df], axis=1)
    precos.index.name="Date"
  return precos


#Data Frame de commodites
df = commodities_df(commodities,inicio,fim)

#Importar dollar
doll = yf.Ticker("USDBRL=x")
dollar = pd.DataFrame(doll.history(start ="2014-01-01", end ="2020-12-31" ).Close)
dollar.columns = ["Dollar"]
dollar.index.name="Date"

#Concatenar dado
dados =pd.merge(df, dollar, how='inner', right_index=True, left_index=True)
dados.head()

,Live Cattle,Feeder Cattle,Dollar
Date,,,
2014-01-01,134.688,NaN,2.3592
2014-01-02,135.625,167.00,2.3571
2014-01-03,136.300,167.62,2.3847
2014-01-06,136.825,168.03,2.3715
2014-01-07,136.525,168.05,2.3737


In [133]:
dados_trimestre = dados.groupby(pd.Grouper(freq='Q')).mean()
dados_trimestre = dados_trimestre.reset_index()
dados_trimestre.head()

,Date,Live Cattle,Feeder Cattle,Dollar
0,2014-03-31,142.532871,171.350492,2.361074
1,2014-06-30,142.672619,190.970476,2.225935
2,2014-09-30,154.548047,220.610000,2.268325
3,2014-12-31,165.997308,233.692187,2.538731
4,2015-03-31,157.177049,210.525246,2.864362


### Dados do SIDRA

In [91]:
import requests

#link do banco de dados sidra
consulta = 'https://apisidra.ibge.gov.br/values/t/1092/n1/all/v/all/p/all/c12716/all/c18/55,56,992,111734,111735/c12529/all/d/v1000151%202,v1000284%202,v1000285%202'
ufs = [5]

#Baixar dados do sidra
df2 = pd.DataFrame()
for uf in ufs:
    url = consulta.format(uf)
    dados_json = requests.get(url).json()
    dados_pandas = pd.DataFrame(dados_json)
    df2 = pd.concat([dados_pandas,df2])
  
#Renomear colunas
sidra = df2.copy() 
colunas = list(df2.iloc[0,:])
sidra.columns = colunas

#Filtrar dados
sidra = sidra.iloc[1:,:]
sidra = sidra[sidra["Variável"]=="Animais abatidos"]
sidra = sidra[sidra["Tipo de inspeção"]=="Total"]
sidra = sidra[sidra["Unidade de Medida"]=="Cabeças"]
sidra = sidra[sidra["Tipo de inspeção"]=="Total"]
sidra = sidra[sidra["Referência temporal"]=="Total do trimestre"]

#Escolher colunas necessárias para a análise
sidra = sidra.reset_index()
sidra = sidra.iloc[:, [5, 11, 15]]

#Dropar coluna de valor X
sidra = sidra.drop(sidra[sidra["Valor"] == 'X'].index)

#tranformar valor em float
sidra["Valor"] = sidra["Valor"].astype(float)
sidra.head()

,Valor,Trimestre,Tipo de rebanho bovino
0,1721468.0,1º trimestre 1997,Bois
1,1173546.0,1º trimestre 1997,Vacas
2,3510085.0,1º trimestre 1997,Total
3,468475.0,1º trimestre 1997,Novilhos
4,142442.0,1º trimestre 1997,Novilhas


In [92]:
#Encontrar datas
times = sidra.Trimestre.str.split(expand=True)
times[2] = pd.to_datetime(times[2])
sidra3 = sidra.copy()
sidra3 = sidra3[times[2] >= "2014-01-01"]
sidra3 = sidra3[times[2] < "2021-01-01"]
sidra3.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Valor,Trimestre,Tipo de rebanho bovino
340,3993691.0,1º trimestre 2014,Bois
341,3067441.0,1º trimestre 2014,Vacas
342,8372872.0,1º trimestre 2014,Total
343,447977.0,1º trimestre 2014,Novilhos
344,863763.0,1º trimestre 2014,Novilhas


In [123]:
#Separar dados pelo tipo de rebanho
bois = sidra3[sidra3["Tipo de rebanho bovino"] == "Bois"]
vacas = sidra3[sidra3["Tipo de rebanho bovino"] == "Vacas"]
novilhas = sidra3[sidra3["Tipo de rebanho bovino"] == "Novilhos"]
novilhos = sidra3[sidra3["Tipo de rebanho bovino"] == "Novilhas"]
total = sidra3[sidra3["Tipo de rebanho bovino"] == "Total"]

#renomear coluna
bois.columns = ["Bois", "Tirmestre", "Tipo de rebanho bovino"]

In [148]:
#Função de contador
def pre_join(dataframe, nome):
  dataframe = dataframe.reset_index()
  dataframe = dataframe.iloc[:, [1]]
  dataframe.columns = [nome]
  return dataframe

#Criar contador de trimestre
bois = pre_join(bois, "Bois")
vacas = pre_join(vacas, "Vacas")
novilhas = pre_join(novilhas, "Novilhas")
novilhos = pre_join(novilhos, "Novilhos")
total = pre_join(total, "Total")

### Juntar os dois datasets

In [151]:
#Função de join
df_final = dados_trimestre.copy()
df_final = df_final.join(bois)
df_final = df_final.join(vacas)
df_final = df_final.join(novilhas)
df_final = df_final.join(novilhos)
df_final = df_final.join(total)
df_final.head()

,Date,Live Cattle,Feeder Cattle,Dollar,Bois,Vacas,Novilhas,Novilhos,Total
0,2014-03-31,142.532871,171.350492,2.361074,3993691.0,3067441.0,447977.0,863763.0,8372872.0
1,2014-06-30,142.672619,190.970476,2.225935,4317627.0,2899909.0,438998.0,882370.0,8538904.0
2,2014-09-30,154.548047,220.610000,2.268325,4627826.0,2561439.0,561864.0,719751.0,8470880.0
3,2014-12-31,165.997308,233.692187,2.538731,4780410.0,2501305.0,565254.0,678093.0,8525062.0
4,2015-03-31,157.177049,210.525246,2.864362,3967926.0,2669421.0,363654.0,738649.0,7739650.0


In [143]:
df_final

,Date,Live Cattle,Feeder Cattle,Dollar
0,2014-03-31,142.532871,171.350492,2.361074
1,2014-06-30,142.672619,190.970476,2.225935
2,2014-09-30,154.548047,220.610000,2.268325
3,2014-12-31,165.997308,233.692187,2.538731
4,2015-03-31,157.177049,210.525246,2.864362
5,2015-06-30,154.694531,218.881905,3.068503
6,2015-09-30,143.911338,207.060313,3.520658
7,2015-12-31,130.377734,173.554063,3.842519
8,2016-03-31,135.051639,158.540984,3.897398
9,2016-06-30,122.838672,147.086508,3.501720


In [ ]:
bois = contador(bois)
vacas = contador(vacas)
novilhas = contador(novilhas)
novilhos = contador(novilhos)
total = contador(total)

##Análises exploratória

###Pré-processamento


### Análises da série temporal

## Modelo de previsão

### Tratamento dos dados

##Conclusão